# Synthetic Views
This example shows how to generate a synthetic view with the help of homographies.
You will see a house from side view and a house from front view.
Please select $n$ point in both images (same order!). The minimum amount of points needed is $n = 4$ but it is very difficult to select exactly the corresponding points in both images. The impreciseness can be seen as noise. Therefore, it's better to build up an overdetermined system ($n > 4$) and to find a good homography approximation.

If you select $n = 6$ you can achieve a pretty good virtual front view.
Please try not to select 3 points being on a line.

In [1]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

%matplotlib tk 
# If Jupyter Notebook gives you errors, please try qt, wx, gtk, osx instead of tk.

img0 = mpimg.imread('14_Chemnitz.jpg')
img1 = mpimg.imread('15_Chemnitz.jpg')

n = 0
while n < 4:
    n_str = input('How many points correspondence do you want to use? Choose n:')
    n = int(n_str)
    if n < 4:
        print("Please choose n >= 4.")

# close old image windows
#plt.close("all")

# select feature points in img0
plt.figure()
plt.imshow(img0)
x_inhom       = np.array(plt.ginput(n));
x_inhom = x_inhom.T

# select feature points in img1
plt.imshow(img1)
x_prime_inhom = np.array(plt.ginput(n));
x_prime_inhom = x_prime_inhom.T
plt.close()

# close new image windows again
print("Input of the first image:")
print(x_inhom)
print("Input of the second image:")
print(x_prime_inhom)


How many points correspondence do you want to use? Choose n:6
Input of the first image:
[[ 536.91527964  533.52883557  848.46813356 1527.45016848 1525.75694644
   865.40035388]
 [ 580.20742817  847.73650925  852.81617535  856.20261941  294.05290472
   483.69377233]]
Input of the second image:
[[ 720.91297043  722.86273522 1210.30393145 1668.49865591 1666.54889113
  1184.95698925]
 [ 553.44024278  853.70401966  843.95519573  843.95519573  557.33977235
   553.44024278]]


Make points homogenous...

In [2]:
x       = np.vstack([x_inhom      , np.ones((1, n), dtype=np.float)])
x_prime = np.vstack([x_prime_inhom, np.ones((1, n), dtype=np.float)])

Remember, the DLT linear equation system of the lecture:  
$\begin{pmatrix}
    \mathbf{0}^T & -w'_i\mathbf{x}^T_i & y'_i\mathbf{x}^T_i \\
    w'_i\mathbf{x}^T_i & \mathbf{0}^T & -x'_i\mathbf{x}^T_i \\
\end{pmatrix} \cdot
\begin{pmatrix}
    \mathbf{h}_{\text{r}1} \\ \mathbf{h}_{\text{r}2} \\ \mathbf{h}_{\text{r}3}
\end{pmatrix}
= \mathbf{0}$  
  
$ \mathbf{A}_i \cdot \mathbf{h} = \mathbf{0} $  
  
Whereas $\mathbf{x}_i$ is the $i$-th point in the point array `x` and 
$\mathbf{x}'_i = \begin{pmatrix} x'_i \\ y'_i \\ w'_i \end{pmatrix}$ is the projectively transformed $i$-th point in the point array `x_prime`.  
$\mathbf{h}_{\text{r}i}$ is `H[i,:].T`, the transposed $i$-th row of the homography $\mathbf{H}$.
  
Now, stack all $\mathbf{A}_i$ for all 4 point correspondences together. Bring all odd rows to the first 4 rows and all even row s to the last 4 rows.

In [3]:
# To avoid squeezing an 2D array with 1 row to a 1D array,
# one can simply use the range i:i+1 (contains only i) instead of a
# single index i.

x_trans = x_prime[0:1,:]
y_trans = x_prime[1:2,:]
w_trans = x_prime[2:3,:]

# Try
#     x_trans = x_prime[0, :]
# to see what happens with
#     x_trans.shape

print(x_trans.shape)

row_1234_col_123 = np.zeros((n, 3), dtype=np.float)
row_1234_col_456 = -w_trans.T * x.T
row_1234_col_789 =  y_trans.T * x.T

row_5678_col_123 = w_trans.T * x.T
row_5678_col_456 = np.zeros((n, 3), dtype=np.float)
row_5678_col_789 = -x_trans.T * x.T

A_top    = np.hstack([row_1234_col_123, row_1234_col_456, row_1234_col_789])
A_bottom = np.hstack([row_5678_col_123, row_5678_col_456, row_5678_col_789])
A        = np.vstack([A_top, A_bottom])
print("shape of A:", A.shape)

(1, 6)
shape of A: (12, 9)


Since $\mathbf{A} \cdot \mathbf{h} \neq \mathbf{0}$, we cannot simply take the right null space of $\mathbf{A}$ but we can decompose $\mathbf{A}$:  
  
$\mathbf{A} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^T$  
  
Then, the *best* $\mathbf{h}$ is the unit singular vector $\mathbf{v}_{\text{r}}$ belonging to the smallest singular value $\mathbf{\sigma}_r$.

In [4]:
U, S, VT = np.linalg.svd(A)
r = np.linalg.matrix_rank(A)

print("rank of DLT system matrix is:", r)
h = VT[r-1, :] # subtract 1 because we start indexing with 0
# VT[r-1, :] is the same as V[:, r-1], the r-th colum vector of V
h = h / h[8]

H_r = h.reshape((3, 3)) # from vector back to matrix
H_r = H_r / H_r[-1, -1] # divide by last row and last collumn to normalize the homography. Now, H[3, 3] should be 1

np.set_printoptions(suppress=True)
print("The reconstructed homography is:")
print(H_r)

rank of DLT system matrix is: 9
The reconstructed homography is:
[[   10.72173838     0.90024569 -3742.35869118]
 [    3.59897239     4.38230685 -2473.11607966]
 [    0.00432073     0.00050105     1.        ]]


Now, let's see if we have transferred the first house image into a front view image...
Plotting is not important for the examination.

In [7]:
scale_output_y = 1.3
scale_output_x = 1.3
out_width = int(img0.shape[1] * scale_output_x)
out_height = int(img0.shape[0] * scale_output_y)
warp_img = cv2.warpPerspective(img0, H_r, (out_width, out_height))

# cv2 uses BGR color channel order -> convert our RGB image to BGR
warp_img_bgr = np.concatenate((warp_img[:,:,2:3], warp_img[:,:,1:2], warp_img[:,:,0:1]), axis=2)

cv2.imwrite("warp.jpg", warp_img_bgr)
#cv2.imwrite("warp.jpg", warp_img)



True